In [1]:
import xarray
import pandas as pd
import numpy as np

In [ ]:
# gfrac = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC_areav2.nc", engine="netcdf4")

In [5]:
GFRAC = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC_NEW.NC", engine="netcdf4")
GFRAC

<xarray.Dataset>
Dimensions:    (time: 27, NGFBFC: 17, latitude: 2160, longitude: 4320)
Coordinates:
  * time       (time) datetime64[ns] 1970-01-01 1975-01-01 ... 2100-01-01
  * latitude   (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * longitude  (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * NGFBFC     (NGFBFC) |S50 b'grass                                         ...
Data variables:
    GFRAC_new  (time, NGFBFC, latitude, longitude) float32 ...

In [3]:
FUBC = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/FUBC_NETCDF.NC", engine="netcdf4")
FUBC

<xarray.Dataset>
Dimensions:       (time: 11, NGFBFC: 14, longitude: 4320, latitude: 2160)
Coordinates:
  * time          (time) datetime64[ns] 1970-01-01 1975-01-01 ... 2020-01-01
  * NGFBFC        (NGFBFC) <U32 'Maize' ... 'Wheat'
  * longitude     (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * latitude      (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
    country       (latitude, longitude) <U40 ...
Data variables:
    N_rate_kg_ha  (time, NGFBFC) float64 ...

In [8]:
gfrac_5min = GFRAC.drop_sel(NGFBFC=b'grass                                             ')
gfrac_5min = gfrac_5min.drop_sel(NGFBFC=b'Oil & palm fruit                               ')
gfrac_5min = gfrac_5min.drop_sel(NGFBFC=b'Temperate oil crops                            ')
gfrac_5min

<xarray.Dataset>
Dimensions:    (time: 27, NGFBFC: 14, latitude: 2160, longitude: 4320)
Coordinates:
  * time       (time) datetime64[ns] 1970-01-01 1975-01-01 ... 2100-01-01
  * latitude   (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * longitude  (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * NGFBFC     (NGFBFC) |S50 b'Wheat                                         ...
Data variables:
    GFRAC_new  (time, NGFBFC, latitude, longitude) float32 ...

In [4]:
FUBC = FUBC.drop("country")
FUBC

<xarray.Dataset>
Dimensions:       (time: 11, NGFBFC: 14, longitude: 4320, latitude: 2160)
Coordinates:
  * time          (time) datetime64[ns] 1970-01-01 1975-01-01 ... 2020-01-01
  * NGFBFC        (NGFBFC) <U32 'Maize' ... 'Wheat'
  * longitude     (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * latitude      (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
Data variables:
    N_rate_kg_ha  (time, NGFBFC) float64 ...

In [11]:
fubc_ngfbfc = [element.strip() for element in FUBC.coords['NGFBFC'].data.astype('str').tolist()]

In [10]:
gfrac_ngfbfc = [element.strip() for element in gfrac_5min.coords['NGFBFC'].data.astype('str').tolist()]

gfrac_5min = gfrac_5min.assign_coords(NGFBFC=gfrac_ngfbfc).isel(time=slice(0, 11))

In [18]:
gfrac_5min["GFRAC_new"].shape

(11, 14, 2160, 4320)

In [19]:
new_data = np.zeros((11, 14, 2160, 4320), dtype='float32')

In [28]:
for i in range(0, 11):
    for x, ngfbfc in enumerate(gfrac_ngfbfc):
        new_data[i][x] = np.multiply(gfrac_5min["GFRAC_new"].isel(time=i).sel(NGFBFC=ngfbfc), FUBC["N_rate_kg_ha"].isel(time=i).sel(NGFBFC=ngfbfc)) / FUBC["N_rate_kg_ha"].isel(time=i).sum(dim='NGFBFC')

In [29]:
new_data_reshape = new_data.reshape((14, 11, 2160, 4320))

In [30]:
newnetcdf = xarray.Dataset(
    coords={
        "time": pd.date_range(start='1970-01-01', end='2020-01-01', freq='5YS'),
        "latitude": GFRAC.coords["latitude"].to_numpy(),
        "longitude": GFRAC.coords["longitude"].to_numpy()
    })
coords = ("time", "latitude", "longitude")
data_vars = {
    ngfbc: (coords, new_data_reshape[i]) for i, ngfbc in enumerate(fubc_ngfbfc)
}
newnetcdf = newnetcdf.assign(data_vars)
newnetcdf

<xarray.Dataset>
Dimensions:                           (time: 11, latitude: 2160, longitude: 4320)
Coordinates:
  * time                              (time) datetime64[ns] 1970-01-01 ... 20...
  * latitude                          (latitude) float64 89.96 89.88 ... -89.96
  * longitude                         (longitude) float64 -180.0 ... 180.0
Data variables: (12/14)
    Maize                             (time, latitude, longitude) float32 nan...
    Other non-food & luxury & spices  (time, latitude, longitude) float32 nan...
    Other temperate cereals           (time, latitude, longitude) float32 nan...
    Plant based fibres                (time, latitude, longitude) float32 nan...
    Pulses                            (time, latitude, longitude) float32 nan...
    Rice                              (time, latitude, longitude) float32 nan...
    ...                                ...
    Temperate roots & tubers          (time, latitude, longitude) float32 nan...
    Tropical cereals                  (time, latitude, longitude) float32 nan...
    Tropical oil crops                (time, latitude, longitude) float32 nan...
    Tropical roots & tubers           (time, latitude, longitude) float32 nan...
    Vegetables & fruits               (time, latitude, longitude) float32 nan...
    Wheat                             (time, latitude, longitude) float32 nan...

In [11]:
gfrac_5min.isel(time=0).sel(NGFBFC='Vegetables & fruits')

<xarray.Dataset>
Dimensions:    (latitude: 2160, longitude: 4320)
Coordinates:
    time       datetime64[ns] 1970-01-01
  * latitude   (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * longitude  (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
    NGFBFC     <U32 'Vegetables & fruits'
Data variables:
    GFRAC_new  (latitude, longitude) float32 ...

In [5]:
FUBC.isel(time=0).sel(NGFBFC='Vegetables & fruits')

<xarray.Dataset>
Dimensions:       (longitude: 4320, latitude: 2160)
Coordinates:
    time          datetime64[ns] 1970-01-01
    NGFBFC        <U32 'Vegetables & fruits'
  * longitude     (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * latitude      (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
    country       (latitude, longitude) <U40 ...
Data variables:
    N_rate_kg_ha  float64 ...

In [ ]:
newnetcdf